minimizing number of steps to process a freshly downloaded IDF curve from climatedata.ca
"Climate Change-Scaled IDF - CMIP6 (ZIP)"

In [2]:
import os
import pandas as pd
import tomllib as tomli
with open("config.toml","rb") as f:
    conf = tomli.load(f)

datahome = os.path.join(conf['folder'], "idf")
datahome

'C:\\Climate-Data\\stjohns\\idf'

In [3]:
histpath = [x for x in os.listdir(datahome) if "historical" in x and x.endswith(".csv")][0]
hist = pd.read_csv(os.path.join(datahome,histpath))
hist

,Duration,2 year,5 year,10 year,25 year,50 year,100 year
0,5 min,56.2,77.0,90.8,108.2,121.1,133.9
1,10 min,41.6,55.7,65.0,76.8,85.6,94.2
2,15 min,34.9,46.9,54.8,64.9,72.3,79.7
3,30 min,24.9,32.8,38.0,44.6,49.5,54.4
4,1 h,17.4,22.3,25.5,29.5,32.5,35.5
5,2 h,11.9,15.8,18.4,21.7,24.1,26.5
6,6 h,6.8,8.5,9.6,11.0,12.0,13.1
7,12 h,4.4,5.3,5.9,6.7,7.3,7.9
8,24 h,2.6,3.1,3.5,3.9,4.3,4.6
9,5 min_95%_confidence_limit,7.2,12.1,16.3,22.0,26.3,30.6


In [4]:
wantscens = conf['scenarios']
hasscens = os.listdir(datahome)
scens = list(set(wantscens).intersection(hasscens))
scens

['ssp245', 'ssp126', 'ssp585']

In [5]:
scenfolder = os.path.join(datahome, scens[1])
os.listdir(scenfolder)


['ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2011-2040.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2021-2050.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2031-2060.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2041-2070.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2051-2080.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2061-2090.csv',
 'ST_JOHN_S_A_8403506_47.62_52.73_ssp126_2071-2100.csv']

In [6]:
prename = os.listdir(scenfolder)[0].split("_ssp")[0]
prename = prename.replace(".","p")

In [13]:
d = {}
for i, scen in enumerate(scens):
    scenfolder = os.path.join(datahome, scen)
    for j, file in enumerate(os.listdir(scenfolder)):
        name = os.path.splitext(file)[0]
        year = name.split("_")[-1]
        title = f"{year}-{scen}"
        data = pd.read_csv(os.path.join(scenfolder, file))
        newheaders = [x for x in data.columns if x.startswith("100 year")] # 100 only
        d[title] = data[newheaders]

In [14]:
output = conf['folder']
outname = f"processed_idf_{prename}.xlsx"

with pd.ExcelWriter(os.path.join(output,outname)) as writer:
    for key in d.keys():
        if not key == "all":
            sheetname = key
            intedata = d[key]
            intedata.to_excel(writer, sheet_name=sheetname, index=False)